In [12]:
import requests
import re
import scrapy
import pandas as pd
import pymongo
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

## Scraping

Various URLs from Earther with different formatting - test cases for the Earther scraper.


In [282]:
url = 'http://earther.gizmodo.com/exxonmobil-wishes-australia-a-happy-new-year-as-country-1840758432'
url = 'https://earther.gizmodo.com/theres-several-orders-of-magnitude-more-plastic-in-rive-1840538231'
url = 'https://deadspin.com/how-did-no-one-notice-this-inspirational-hiker-on-the-p-1818647235'
url = 'https://jalopnik.com/its-time-to-let-go-of-commuter-culture-1840630621'
url = 'https://earther.gizmodo.com/climate-change-stole-the-show-at-the-golden-globes-1840830510'
response = requests.get(url)
scrap = scrapy.http.HtmlResponse(url=url, 
                                 request=scrapy.http.Request(url), body=response.text, encoding='utf-8')

The Earther webcrawler added 3015 articles to the MongoDB (articles collection in items).

In [284]:
url = 'https://www.dailykos.com/news/Climate'
response = requests.get(url)
scrap = scrapy.http.HtmlResponse(url=url, 
                                 request=scrapy.http.Request(url), body=response.text, encoding='utf-8')

In [300]:
scrap.css(".nav-wrapper a::attr(href)").get()

'/part/story/table/by_tag?tag_id=35567&page=1'

## Load data from Mongo

In [5]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
mydb = myclient['items']

In [6]:
mycollection = mydb['articles']

In [7]:
df = pd.DataFrame(list(mycollection.find({})))
df.created_at = pd.to_datetime(df.created_at, infer_datetime_format=True)
df.head()

,_id,url,title,twitter_url,image,keywords,description,num_like,num_reply,author,author_link,created_at,body_text,body_links
0,5e14bee3f106fd495904e5c5,https://earther.gizmodo.com/the-worlds-largest...,The World's Largest Floating Wind Farm Is Here,https://earther.gizmodo.com/the-worlds-largest...,https://i.kinja-img.com/gawker-media/image/upl...,"[Portugal, Earther, Wind Energy, Wind for the ...","This is the second day of the new decade, and ...",13000,34,Yessenia Funes,https://kinja.com/yessfun,2020-01-02 17:00:00-05:00,"This is the second day of the new decade, and ...",[http://www.principlepowerinc.com/en/news-pres...
1,5e14bee3f106fd495904e5c6,https://earther.gizmodo.com/trumps-tremendous-...,Trump's 'Tremendous' Superfund Work Includes N...,https://earther.gizmodo.com/trumps-tremendous-...,https://i.kinja-img.com/gawker-media/image/upl...,"[EPA, Donald Trump, Earther, Toxic Legacy, Env...",The Environmental Protection Agency under Dona...,9200,16,Yessenia Funes,https://kinja.com/yessfun,2020-01-03 15:30:00-05:00,The Environmental Protection Agency under Dona...,[https://earther.gizmodo.com/the-epas-superfun...
2,5e14bee3f106fd495904e5c7,https://lifehacker.com/how-to-help-those-affec...,How to Help Those Affected By the Australian W...,https://lifehacker.com/how-to-help-those-affec...,https://i.kinja-img.com/gawker-media/image/upl...,"[Lifehacker, AUSTRALIA, call for help, Fire]","As of Thursday morning, ongoing bushfires have...",15100,12,Josh Ocampo,https://kinja.com/joshocampo,2020-01-02 14:45:00-05:00,"As of Thursday morning, ongoing bushfires have...",[https://www.theguardian.com/australia-news/li...
3,5e14bee3f106fd495904e5c8,https://earther.gizmodo.com/smoke-from-austral...,"Smoke From Australia’s Horrific Wildfires, as ...",https://earther.gizmodo.com/smoke-from-austral...,https://i.kinja-img.com/gawker-media/image/upl...,"[fires, global warming, Satellite Images, heat...",Significant portions of Australia’s southeast ...,38700,15,George Dvorsky,https://kinja.com/georgedvorsky,2020-01-03 10:10:00-05:00,Significant portions of Australia’s southeast ...,[https://earthobservatory.nasa.gov/images/1460...
4,5e14bee3f106fd495904e5c9,https://earther.gizmodo.com/earthquake-shows-h...,Earthquake Shows Puerto Rico's Electric Grid I...,https://earther.gizmodo.com/earthquake-shows-h...,https://i.kinja-img.com/gawker-media/image/upl...,"[Puerto Rico, Hurricane Maria, Earther, Energy...",Puerto Rico just can’t seem to catch a break. ...,8100,6,Yessenia Funes,https://kinja.com/yessfun,2020-01-06 15:55:00-05:00,Puerto Rico just can’t seem to catch a break. ...,[http://ds.iris.edu/seismon/quakes_today_in_N_...


In [8]:
keywords = df.keywords.sum()

In [366]:
keyword_count = pd.Series(keywords).str.lower().str.replace(" ","").value_counts()
keyword_count[keyword_count > 2]

earther                    2536
climatechange               668
science                     527
gizmodo                     294
conservation                268
                           ... 
lymedisease                   3
nuclearwaste                  3
fiatchrysler                  3
hurricanemariacontinues       3
explainers                    3
Length: 856, dtype: int64

In [340]:
pd.set_option('display.max_rows', 100)
pd.Series(keywords).str.lower().str.replace(" ","").value_counts().head(100)

earther                              2536
climatechange                         668
science                               527
gizmodo                               294
conservation                          268
environmentaljustice                  237
weather                               173
climate                               160
wildfires                             128
california                            117
epa                                   105
environment                           103
wildlife                              103
trumpadministration                    96
weatherishappening                     77
ecology                                77
antarctica                             76
naturaldisasters                       75
airpollution                           73
globalwarming                          72
animals                                72
hurricanemaria                         71
puertorico                             69
pollution                         

In [341]:
import nltk
from nltk.corpus import stopwords
import string
from nltk import word_tokenize, FreqDist


In [355]:
stopwords_list = stopwords.words('english') + list(string.punctuation) + ["''", '""', '...', '``','’','“','”']

def process_article(article):
    tokens = nltk.word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stopwords_list]
    return stopwords_removed    

In [356]:
df['tokens'] = df.body_text.map(process_article)

In [348]:
df.head()

,_id,url,title,twitter_url,image,keywords,description,num_like,num_reply,author,author_link,created_at,body_text,body_links,tokens
0,5e14bee3f106fd495904e5c5,https://earther.gizmodo.com/the-worlds-largest...,The World's Largest Floating Wind Farm Is Here,https://earther.gizmodo.com/the-worlds-largest...,https://i.kinja-img.com/gawker-media/image/upl...,"[Portugal, Earther, Wind Energy, Wind for the ...","This is the second day of the new decade, and ...",13000,34,Yessenia Funes,https://kinja.com/yessfun,2020-01-02T17:00:00-05:00,"This is the second day of the new decade, and ...",[http://www.principlepowerinc.com/en/news-pres...,"[second, day, new, decade, world, largest, flo..."
1,5e14bee3f106fd495904e5c6,https://earther.gizmodo.com/trumps-tremendous-...,Trump's 'Tremendous' Superfund Work Includes N...,https://earther.gizmodo.com/trumps-tremendous-...,https://i.kinja-img.com/gawker-media/image/upl...,"[EPA, Donald Trump, Earther, Toxic Legacy, Env...",The Environmental Protection Agency under Dona...,9200,16,Yessenia Funes,https://kinja.com/yessfun,2020-01-03T15:30:00-05:00,The Environmental Protection Agency under Dona...,[https://earther.gizmodo.com/the-epas-superfun...,"[environmental, protection, agency, donald, tr..."
2,5e14bee3f106fd495904e5c7,https://lifehacker.com/how-to-help-those-affec...,How to Help Those Affected By the Australian W...,https://lifehacker.com/how-to-help-those-affec...,https://i.kinja-img.com/gawker-media/image/upl...,"[Lifehacker, AUSTRALIA, call for help, Fire]","As of Thursday morning, ongoing bushfires have...",15100,12,Josh Ocampo,https://kinja.com/joshocampo,2020-01-02T14:45:00-05:00,"As of Thursday morning, ongoing bushfires have...",[https://www.theguardian.com/australia-news/li...,"[thursday, morning, ongoing, bushfires, contin..."
3,5e14bee3f106fd495904e5c8,https://earther.gizmodo.com/smoke-from-austral...,"Smoke From Australia’s Horrific Wildfires, as ...",https://earther.gizmodo.com/smoke-from-austral...,https://i.kinja-img.com/gawker-media/image/upl...,"[fires, global warming, Satellite Images, heat...",Significant portions of Australia’s southeast ...,38700,15,George Dvorsky,https://kinja.com/georgedvorsky,2020-01-03T10:10:00-05:00,Significant portions of Australia’s southeast ...,[https://earthobservatory.nasa.gov/images/1460...,"[significant, portions, australia, southeast, ..."
4,5e14bee3f106fd495904e5c9,https://earther.gizmodo.com/earthquake-shows-h...,Earthquake Shows Puerto Rico's Electric Grid I...,https://earther.gizmodo.com/earthquake-shows-h...,https://i.kinja-img.com/gawker-media/image/upl...,"[Puerto Rico, Hurricane Maria, Earther, Energy...",Puerto Rico just can’t seem to catch a break. ...,8100,6,Yessenia Funes,https://kinja.com/yessfun,2020-01-06T15:55:00-05:00,Puerto Rico just can’t seem to catch a break. ...,[http://ds.iris.edu/seismon/quakes_today_in_N_...,"[puerto, rico, seem, catch, break, people, isl..."


In [357]:
total_vocab = set()
for article in df.tokens:
    total_vocab.update(article)
len(total_vocab)

63789

In [358]:
articles_concat = []
for article in df.tokens:
    articles_concat += article

In [359]:
articles_freqdist = FreqDist(articles_concat)

In [376]:
articles_freqdist.most_common(50)

[('climate', 8477),
 ('said', 5923),
 ('new', 5678),
 ('also', 5127),
 ('change', 4960),
 ('like', 4806),
 ('one', 4644),
 ('people', 4628),
 ('could', 4502),
 ('would', 3840),
 ('water', 3700),
 ('year', 3541),
 ('told', 3468),
 ('world', 3309),
 ('earther', 3115),
 ('even', 2993),
 ('time', 2872),
 ('years', 2870),
 ('last', 2676),
 ('according', 2633),
 ('still', 2546),
 ('ice', 2472),
 ('state', 2434),
 ('much', 2361),
 ('study', 2351),
 ('around', 2307),
 ('environmental', 2281),
 ('percent', 2244),
 ('first', 2222),
 ('may', 2176),
 ('species', 2170),
 ('energy', 2164),
 ('many', 2146),
 ('get', 2134),
 ('u.s.', 2133),
 ('carbon', 2133),
 ('way', 2126),
 ('national', 2045),
 ('research', 1962),
 ('scientists', 1959),
 ('make', 1891),
 ('sea', 1868),
 ('air', 1835),
 ('well', 1804),
 ('back', 1791),
 ('since', 1778),
 ('two', 1769),
 ('see', 1737),
 ('emissions', 1725),
 ('need', 1686)]

In [383]:
df.keywords.map(lambda lst : ' '.join([k.replace(' ','-') for k in lst]))

0       Portugal Earther Wind-Energy Wind-for-the-Win ...
1       EPA Donald-Trump Earther Toxic-Legacy Environm...
2                 Lifehacker AUSTRALIA call-for-help Fire
3       fires global-warming Satellite-Images heat-wav...
4       Puerto-Rico Hurricane-Maria Earther Energy Ear...
                              ...                        
3010                                              Earther
3011    Puerto-Rico El-Yunque hurricanes science disas...
3012                     Lifehacker Pets Environment Dogs
3013    polar science Earther art sound-of-climate-sil...
3014    whales science Earther conservation marine-bio...
Name: keywords, Length: 3015, dtype: object

## K-Means Clustering on keywords

First, we need to make a sparse array of keywords (1 column per keyword, 1 row per article).

In [10]:
key_df = pd.DataFrame(df.keywords.copy())

In [11]:
key_df

,keywords
0,"[Portugal, Earther, Wind Energy, Wind for the ..."
1,"[EPA, Donald Trump, Earther, Toxic Legacy, Env..."
2,"[Lifehacker, AUSTRALIA, call for help, Fire]"
3,"[fires, global warming, Satellite Images, heat..."
4,"[Puerto Rico, Hurricane Maria, Earther, Energy..."
...,...
3010,[Earther]
3011,"[Puerto Rico, El Yunque, hurricanes, science, ..."
3012,"[Lifehacker, Pets, Environment, Dogs]"
3013,"[polar, science, Earther, art, sound of climat..."


In [13]:
vect = CountVectorizer()

key_df = pd.DataFrame(df.keywords.copy())

X = vect.fit_transform(key_df.pop('keywords').map(lambda lst : ' '.join([k.replace(' ','_') for k in lst])))

for i, col in enumerate(vect.get_feature_names()):
    key_df[col] = pd.Series(X[:, i].toarray().ravel())


In [14]:
key_df.loc[1481][key_df.loc[1481] > 0]

anniversary             1
best_of_earther         1
best_of_gizmodo         1
earther                 1
earther_is_best_blog    1
Name: 1481, dtype: int64

In [15]:
df.loc[1481].keywords

['Earther',
 'anniversary',
 'best of gizmodo',
 'earther is best blog',
 'best of earther']

NOTE: We may want to drop all "Earther", "Gizmodo", "Jalopnik", etc. keywords.

Next, we perform k-means clustering!

In [16]:
k_means = KMeans(n_clusters = 10)
k_means.fit(key_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [17]:
k_means.predict(key_df)

array([8, 8, 0, ..., 0, 6, 5], dtype=int32)

In [18]:
df['cluster'] = k_means.predict(key_df)

In [22]:
pd.Series(df[df['cluster'] == 1].keywords.sum()).value_counts().head(40)

Earther                         227
Environmental Justice           195
environmental justice            33
Climate Change                   23
Flint Water Crisis               22
Hurricane Maria                  22
Puerto Rico                      20
EPA                              19
Lead                             18
Air Pollution                    16
Trump Administration             13
Bayou Bridge Pipeline            10
Keystone XL                       8
Health                            8
Goddamn Pollution                 8
Indigenous Rights                 8
Oil and Gas                       8
Hurricane Harvey                  7
Pollution                         7
Louisiana                         7
Standing Rock                     7
climate change                    6
Bears Ears National Monument      6
Art                               6
Dakota Access Pipeline            6
Oil Pipelines                     5
Environment                       5
Natural Gas                 

In [23]:
pd.Series(df[df['cluster'] == 0].keywords.sum()).value_counts().head(40)

Gizmodo                      70
Jalopnik                     43
Lifehacker                   41
climate change               27
Jezebel                      26
Climate Change               17
The Root                     17
environment                  16
Environment                  13
The Slot                     12
Deadspin                     12
donald trump                  9
car policy                    8
News                          8
animals                       8
recycling                     8
recycle                       7
technology                    7
EPA                           7
Flint water crisis            7
Vitals                        6
puerto rico                   5
Puerto Rico                   5
automaton                     5
epa                           5
environmental racism          5
natural disasters             5
Flint Michigan                5
Donald Trump                  5
Hurricane Maria               5
Flint water contamination     5
Technolo

In [45]:
pd.Series(df[df['cluster'] == 1].keywords.sum()).value_counts().head(20).index.to_list()

['Earther',
 'Environmental Justice',
 'environmental justice',
 'Climate Change',
 'Flint Water Crisis',
 'Hurricane Maria',
 'Puerto Rico',
 'EPA',
 'Lead',
 'Air Pollution',
 'Trump Administration',
 'Bayou Bridge Pipeline',
 'Keystone XL',
 'Health',
 'Goddamn Pollution',
 'Indigenous Rights',
 'Oil and Gas',
 'Hurricane Harvey',
 'Pollution',
 'Louisiana']

Excellent, we have generated meaningful categories.

In [66]:
pd.Series(df[df['cluster'] == i].keywords.sum()).value_counts().head(20).reset_index()[['index',0]].apply(lambda x : x.astype(str)).apply(" ".join, axis=1)

0                               Earther 73
1                             wildfires 51
2     wildfire season is year round now 41
3                            California 30
4                            california 22
5                             Wildfires 18
6                        climate change 17
7                  California wildfires 14
8                             camp fire 13
9                         Climate Change 8
10                               weather 6
11                                  PG&E 5
12                             Camp Fire 5
13        california can't catch a break 5
14                              paradise 4
15                          woolsey fire 4
16                       Extreme Weather 3
17                               climate 3
18                                 smoke 3
19                   paradise california 3
dtype: object

In [67]:
clusters = {}
for i in range(10):
    clusters[i] = pd.Series(df[df['cluster'] == i].keywords.sum()).value_counts().head(20).reset_index()[['index',0]].apply(lambda x : x.astype(str)).apply(" ".join, axis=1)
pd.DataFrame(clusters)

,0,1,2,3,4,5,6,7,8,9
0,Gizmodo 70,Earther 227,Earther 502,Earther 54,Gizmodo 16,Earther 219,Earther 259,Gizmodo 207,Earther 1201,Earther 73
1,Jalopnik 43,Environmental Justice 195,climate change 325,climate policy 32,science 11,conservation 173,science 110,science 166,weather is happening 46,wildfires 51
2,Lifehacker 41,environmental justice 33,Climate Change 174,2020 election 20,Volcanoes 8,Conservation 48,climate 95,Science 43,weather 41,wildfire season is year round now 41
3,climate change 27,Climate Change 23,Science 43,2020 Election 9,volcanoes 6,wildlife 48,Science 76,biology 28,Air Pollution 31,California 30
4,Jezebel 26,Flint Water Crisis 22,climate 40,green new deal 7,earth sciences 4,science 28,weather 44,Climate Change 22,hurricane season 29,california 22
5,Climate Change 17,Hurricane Maria 22,science 35,jay inslee 6,geology 4,animals 22,weather is happening 17,climate change 18,natural disasters 29,Wildfires 18
6,The Root 17,Puerto Rico 20,ice on thin ice 32,renewable energy 6,volcanic eruptions 3,ecology 20,wild weather 16,Conservation 17,EPA 27,climate change 17
7,environment 16,EPA 19,weather 30,trump administration 3,Mount kilauea 3,marine biology 12,hurricane season 14,Environment 17,they call it global warming for a reason 27,California wildfires 14
8,Environment 13,Lead 18,arctic 26,Trump Administration 3,Disasters 3,invasive species 12,climate change 12,geology 13,wildlife 26,camp fire 13
9,The Slot 12,Air Pollution 16,sea level rise 22,Elizabeth Warren 3,Seismology 3,climate change 10,ecology 11,birds 12,oceans 25,Climate Change 8


In [70]:
df.cluster.value_counts().sort_index()

0     248
1     228
2     502
3      54
4      16
5     220
6     260
7     210
8    1201
9      76
Name: cluster, dtype: int64

Note that the first category is of cross-posted articles. That's cool that they're being found but we should drop those keywords and re-cluster.

The other categories we've found: 
1. EJ / eco-activism
1. climate science
1. climate politics
1. volcanoes & earthquakes (tectonics)
1. conservation
1. science of extreme weather
1. biology
1. extreme weather / global warming is real / catchall
1. wildfires

Note that categories 6 and 8 seem very overlapping. Also the categories are of very different sizes, which isn't ideal.

Do we have the right clusters? Need to do the silhouette score/elbowing.
* make sure to build process that allows for new classes / different classes to be generated
* build the pipeline
* how do you scrape to just add new stuff? have a meta table that keeps track of the last run, stop once your date is before. If articles have sequential ids you could in theory use that

use rss feeds follow the link, grab ALL the text then use tfidf to grab only the interesting words

In [71]:
df.cluster.max()

9

In [182]:
import importlib
import topic_modeler.modeling
importlib.reload(topic_modeler.modeling)
from topic_modeler.modeling import Modeler

In [183]:
modeler = Modeler()

In [138]:
modeler.articles

,_id,url,title,twitter_url,image,keywords,description,num_like,num_reply,author,author_link,created_at,body_text,body_links
0,5e14bee3f106fd495904e5c5,https://earther.gizmodo.com/the-worlds-largest...,The World's Largest Floating Wind Farm Is Here,https://earther.gizmodo.com/the-worlds-largest...,https://i.kinja-img.com/gawker-media/image/upl...,"[Portugal, Earther, Wind Energy, Wind for the ...","This is the second day of the new decade, and ...",13000,34,Yessenia Funes,https://kinja.com/yessfun,2020-01-02 17:00:00-05:00,"This is the second day of the new decade, and ...",[http://www.principlepowerinc.com/en/news-pres...
1,5e14bee3f106fd495904e5c6,https://earther.gizmodo.com/trumps-tremendous-...,Trump's 'Tremendous' Superfund Work Includes N...,https://earther.gizmodo.com/trumps-tremendous-...,https://i.kinja-img.com/gawker-media/image/upl...,"[EPA, Donald Trump, Earther, Toxic Legacy, Env...",The Environmental Protection Agency under Dona...,9200,16,Yessenia Funes,https://kinja.com/yessfun,2020-01-03 15:30:00-05:00,The Environmental Protection Agency under Dona...,[https://earther.gizmodo.com/the-epas-superfun...
2,5e14bee3f106fd495904e5c7,https://lifehacker.com/how-to-help-those-affec...,How to Help Those Affected By the Australian W...,https://lifehacker.com/how-to-help-those-affec...,https://i.kinja-img.com/gawker-media/image/upl...,"[Lifehacker, AUSTRALIA, call for help, Fire]","As of Thursday morning, ongoing bushfires have...",15100,12,Josh Ocampo,https://kinja.com/joshocampo,2020-01-02 14:45:00-05:00,"As of Thursday morning, ongoing bushfires have...",[https://www.theguardian.com/australia-news/li...
3,5e14bee3f106fd495904e5c8,https://earther.gizmodo.com/smoke-from-austral...,"Smoke From Australia’s Horrific Wildfires, as ...",https://earther.gizmodo.com/smoke-from-austral...,https://i.kinja-img.com/gawker-media/image/upl...,"[fires, global warming, Satellite Images, heat...",Significant portions of Australia’s southeast ...,38700,15,George Dvorsky,https://kinja.com/georgedvorsky,2020-01-03 10:10:00-05:00,Significant portions of Australia’s southeast ...,[https://earthobservatory.nasa.gov/images/1460...
4,5e14bee3f106fd495904e5c9,https://earther.gizmodo.com/earthquake-shows-h...,Earthquake Shows Puerto Rico's Electric Grid I...,https://earther.gizmodo.com/earthquake-shows-h...,https://i.kinja-img.com/gawker-media/image/upl...,"[Puerto Rico, Hurricane Maria, Earther, Energy...",Puerto Rico just can’t seem to catch a break. ...,8100,6,Yessenia Funes,https://kinja.com/yessfun,2020-01-06 15:55:00-05:00,Puerto Rico just can’t seem to catch a break. ...,[http://ds.iris.edu/seismon/quakes_today_in_N_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,5e14bf64f106fd495904f187,https://earther.gizmodo.com/about-earther-1818...,About Earther,https://earther.gizmodo.com/about-earther-1818...,https://i.kinja-img.com/gawker-media/image/upl...,[Earther],"Thanks for stopping by Earther, a destination ...",5800,0,Earther Staff,NaN,2017-09-27 08:00:00-04:00,"Thanks for stopping by Earther, a destination ...",[https://gizmodo.com/gizmodo-media-groups-edit...
3011,5e14bf64f106fd495904f188,https://earther.gizmodo.com/americas-only-trop...,The Only Tropical Rainforest in the National F...,https://earther.gizmodo.com/americas-only-trop...,https://i.kinja-img.com/gawker-media/image/upl...,"[Puerto Rico, El Yunque, hurricanes, science, ...",A week after Hurricane Maria tore through the ...,24300,61,Maddie Stone,https://kinja.com/maddiestone,2017-09-27 15:00:00-04:00,A week after Hurricane Maria tore through the ...,[https://www.nytimes.com/2017/09/25/us/puerto-...
3012,5e14bf64f106fd495904f189,https://lifehacker.com/how-to-keep-your-dog-fr...,How to Keep Your Dog from Destroying the Envir...,https://lifehacker.com/how-to-keep-your-dog-fr...,https://i.kinja-img.com/gawker-media/image/upl...,"[Lifehacker, Pets, Environment, Dogs]","Dogs are humans’ best friends, immeasurable br...",20600,53,Tim Donnelly,https://kinja.com/timdo

In [153]:
modeler.gizmodo_stop_words

['earther',
 'earther_is_a_berg_blog_now',
 'paleofuture',
 'earther_is_a_scat_blog_now',
 'best_of_earther',
 'the_root',
 'sorry_but_earther_is_a_cats_blog_now',
 'lifehacker',
 'earther_is_a_real_estate_blog_now',
 'earther_is_a_beauty_blog_now',
 'earther_is_an_art_blog_now',
 'best_of_gizmodo',
 'deadspin',
 'earther_is_a_car_blog_now',
 'earther_is_a_poop_blog_now',
 'welcome_to_earther',
 'earther_is_a_food_blog_now',
 'the_muse',
 'earther_is_a_sports_blog_now',
 'the_takeout',
 'earther_is_a_royal_baby_blog_now',
 'the_slot',
 'earther_is_a_youth_interest_blog',
 'earther_is_an_earth_blog',
 'earther_is_best_blog',
 'jalopnik',
 'the_concourse',
 'jalopnik_investigates',
 'earther_is_a_crypto_blog_now',
 'flat_dot_earther_dot_com',
 'earther',
 'earther_is_a_gaming_blog_now',
 'earther_is_a_hog_blog_now',
 'gizmodo',
 'earther_goes_to_the_museum',
 'jezebel',
 'earther_is_a_gamer_blog_now']

In [157]:
modeler.vectorized_keywords

Keywords have not been vectorized. Run Modeler.vectorize_keywords().


In [184]:
modeler.vectorize_keywords()

,11,13,17,18_months_until_take_off,2010s,2011_japan_earthquake,2016_election,2017_hurricane_season,2017_in_review,2018,...,zero_hour,zero_waste,zone,zoo,zoo_knoxville,zoology,zoomers,zoos,zsolt_török,ʻōhiʻa
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [185]:
modeler.cluster_keywords(10)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [186]:
modeler.display_keyword_clusters()

,0,1,2,3,4,5,6,7,8,9
0,Survival Skills 1,wildfires 42,science 250,weather is happening 47,science 72,weather 128,climate change 288,conservation 151,Trump Administration 61,Environmental Justice 164
1,Heat 1,climate 34,Science 94,climate 13,climate change 65,climate 56,Climate Change 163,Conservation 48,trump administration 33,environmental justice 24
2,Heat Week 1,wildfire season is year round now 32,conservation 30,snow 8,Climate Change 65,Weather 36,ice on thin ice 27,wildlife 48,climate change 17,Hurricane Maria 22
3,Outdoors 1,natural disasters 31,biology 27,typhoons 6,Science 60,climate change 32,wildfires 25,animals 19,Environmental Justice 12,Flint Water Crisis 22
4,Summer 1,Air Pollution 31,ecology 22,climate change 5,Environment 16,science 27,climate 25,ecology 17,climate policy 10,Puerto Rico 20
5,Survival 1,California 31,geology 21,hurricanes 5,global warming 14,weather is happening 26,arctic 20,invasive species 11,EPA 10,Lead 18
6,Wilderness 1,renewable energy 28,marine biology 15,heat wave 5,climate 11,wild weather 17,Environmental Justice 19,climate change 9,Climate Change 10,Air Pollution 16
7,NaN,they call it global warming for a reason 27,birds 14,hurricane season 5,Conservation 10,hurricane season 14,sea level rise 18,environment 8,border wall 5,EPA 15
8,NaN,EPA 27,animals 14,flooding 5,wildfires 9,meteorology 11,California 18,ryan zinke 8,fossil fuels 4,Bayou Bridge Pipeline 10
9,NaN,ice on thin ice 26,Conservation 13,tornadoes 4,environment 8,Meteorology 10,climate policy 16,endangered species 7,Oil and Gas 4,Indigenous Rights 8
